### Visualize camera frames at each timestamp 

In [ ]:
import os
import cv2

BASE_DIR = "generated_data"
CAM_DIRS = ["CAM_FRONT", "CAM_FRONT_LEFT", "CAM_FRONT_RIGHT", "CAM_BACK", "CAM_BACK_LEFT", "CAM_BACK_RIGHT"]

# Read all filenames in the target directory
filenames = os.listdir(os.path.join(BASE_DIR, CAM_DIRS[0]))

# Extract the filename without the extension
timestamps = set([os.path.splitext(filename)[0] for filename in filenames])

for timestamp in sorted(timestamps):
    for cam_dir in CAM_DIRS:
        camera_image_filename = timestamp + ".png"
        camera_image_path = os.path.join(BASE_DIR, cam_dir, camera_image_filename)
        camera_image = cv2.imread(camera_image_path)
        camera_image = cv2.resize(camera_image, (400, 225))
        cv2.imshow(cam_dir, camera_image)
    if cv2.waitKey(0) == ord('q'):
        break
cv2.destroyAllWindows()

### Visualize pointclouds at each timestamp

In [ ]:
import open3d as o3d
import numpy as np

# Load the point cloud from a PLY file
import os 


current_dir = os.getcwd()

data_dir = os.path.join(current_dir, 'generated_data_small', 'DEPTH_BEV')

stuff = []

for file in os.listdir(data_dir):
    file_path = os.path.join(data_dir, file)
    
    # Check if the file is a .npy file
    
    print(file_path)
    if file.endswith('.npy'):
        matrix = np.load(file_path)
        """lh_matrix = np.array([
                    [1, 0, 0, 0],
                    [0, -1, 0, 0],  # Flip y-axis
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]
                ])
        matrix = lh_matrix @ matrix"""
                
        ply_file = file_path.replace('.npy', '.ply')
        point_cloud = o3d.io.read_point_cloud(ply_file)
        point_cloud.transform(matrix)
        
        # create axis frame
        axis_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.9, origin=[0, 0, 0])
        axis_frame.transform(matrix)
        
        stuff += [point_cloud, axis_frame]
        
                
o3d.visualization.draw_geometries(stuff)

### Visualize transform positions and orientations with labels

In [ ]:
import os
import numpy as np
import open3d as o3d

# Get the current directory
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, 'src', 'generated_data')

# Create an Open3D visualization window
vis = o3d.visualization.Visualizer()
vis.create_window()


filename = None
# Iterate over all subdirectories in the current directory
for folder in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder)
    
    # Check if the item is a directory
    if os.path.isdir(folder_path):
        # Look for .npy files in the directory
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            
            # Check if the file is a .npy file
            if file.endswith('.npy'):
                if filename is None:
                    filename = os.path.basename(file_path)
  
                
                # Load the .npy file as a 4x4 transformation matrix
                # print(file_path)
                file_path = os.path.join(folder_path, filename)
                matrix = np.load(file_path)
                
                lh_matrix = np.array([
                    [1, 0, 0, 0],
                    [0, -1, 0, 0],  # Flip y-axis
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]
                ])
                matrix = lh_matrix @ matrix
                
                # Create an Open3D geometry from the transformation matrix
                frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2, origin=[0, 0, 0])
                frame.transform(matrix)
                
                # Add the frame to the visualization
                vis.add_geometry(frame)
                
                # Create a text label for the folder name
                text_mesh = o3d.t.geometry.TriangleMesh.create_text(folder, depth=0.1).to_legacy()
                text_mesh.paint_uniform_color((1, 0, 0))
                
                # Scale down and position the text
                scale = 0.005
                text_mesh.transform([[scale, 0, 0, matrix[0, 3]], 
                                     [0, scale, 0, matrix[1, 3]], 
                                     [0, 0, scale, matrix[2, 3]], 
                                     [0, 0, 0, 1]])
                
                # Enable back-face rendering
                text_mesh.compute_vertex_normals()
                vis.get_render_option().mesh_show_back_face = True
                
                # Add the text label to the visualization
                vis.add_geometry(text_mesh)

# Run the visualization
vis.run()

# Close the visualization window
vis.destroy_window()